In [3]:
import numpy as np
import cv2

In [4]:
class conv:
    def __init__(self,filter,bias):
        # F.shape=(C*FH*FW,FN)
        self.F=filter
        # b.shape=(1,FN)
        self.b=bias
        # data.shape=(N*OH*OW,FN)
        self.data=None
    def forward(self,data,batch_size,FH,FW,channel=3,stride=1,padding=0):
        # ready2dot: (N*H,W*C) => (N*OH*OW,C*FH*FW)
        self.data=ready2dot(data,stride,padding,FH,FW,channel,batch_size)
        ret=np.dot(data,self.F)+self.b
        return ret
    def backward(self,diff_y,learning_rate=0.1):
        F-=learning_rate*np.dot(self.data.T,diff_y)
        f=lambda x: x if len(x[0])==1 else x[0]+f(x[1:])
        temp=f(diff_y)/diff_y.shape[0]
        b-=learning_rate*temp
        return np.dot(diff_y,self.F.T)

In [5]:
def ready2dot(data,stride,padding,FH,FW,C,N):
    # data.shape=(N*H,W*C)
    # add padding
    data=np.concatenate((np.zeros((W*C,padding),dtype=int),data),axis=1)
    data=np.concatenate((data,np.zeros((W*C,padding),dtype=int)),axis=1)
    data=np.concatenate(np.zeros((padding,N*H+2*padding),dtype=int),data,axis=0)
    data=np.concatenate(data,np.zeros((padding,N*H+2*padding),dtype=int),axis=0)
    OH=N*int((H-FH+2*padding)/stride)+1
    OW=N*int((W-FW+2*padding)/stride)+1
    ret=[]
    for i in range(OH):
        temp=data[i*stride:i*stride+FH]
        for j in range(OW):
            L=temp[j*stride*C:j*stride*C+C*FW].reshape(1,C*FH*FW)
            ret.append(L)
    return ret

In [6]:
class relu:
    def __init__(self):
        self.mask=None
    def forward(self,data):
        self.mask=(data<=0)
        ret=data.copy()
        ret[self.mask]=0
        return ret
    def backward(self,diff_y):
        diff_y[self.mask]=0
        return diff_y

In [7]:
class pooling:
    def __init__(self,stride):
        self.stride=stride
        self.mask=[]
    def forward(self,data):
        ret=[]
        H,W=data.shape
        OH=int(H/self.stride)
        OW=int(W/self.stride)
        for i in range(OH):
            temp=data[i*self.stride:(i+1)*self.stride]
            for j in range(OW):
                tmp2=temp[j*self.stride:(j+1)*self.stride]
                max_val=max(tmp2)
                # mask,shape=((H/s)*(W/s),s,s)
                self.mask.append(tmp2<max_val)
                ret.append(max_val)
        self.mask.reshape(H,W)
        return ret.reshape(OH,OW)
    def backward(self,diff_y):
        H,W=self.mask.shape
        OH=int(H/self.stride)
        OW=int(W/self.stride)
        ret=np.ones(H,W)
        for i in range(OH):
            tmp=self.mask[i*self.stride:(i+1)*self.stride]
            for j in range(OW):
                tmp2=tmp[j*self.stride:(j+1)*self.stride]
                ret[tmp2]=0
                ret[not tmp2]=diff_y[i][j]
        return ret

In [2]:
class Affine:
    def __init__(self,W,b):
        # W.shape=(M,O)
        self.W=W
        # b.shape=(1,O)
        self.b=b
    def forward(self,batch):
        #batch.shape=(N,M)
        ret=np.dot(batch,W)+b
        return ret
    def backward(self,diff_y,learning_rate=0.1):
        self.b-=learning_rate*diff_y
        self.W-=learning_rate*np.dot(x.T,diff_y)
        return np.dot(diff_y,self.W.T)

In [ ]:
def Loss(y,t):
    # t: predict, y: answer
    # t.shape=(N*H*W,1)
    